In [ ]:
# Install required packages
!pip install -q opencv-python-headless requests

import cv2
import os
from datetime import timedelta
import shutil
import zipfile
from google.colab import files
import time

# --- Configuration ---
# Set the frame interval to 0.5 for half-second extraction
frame_interval_seconds = 0.5
output_dir = "temp_frames"
zip_output = "extracted_frames.zip"

# Step 1: Upload video from your local machine
def upload_video():
    print("Please choose your video file to upload.")
    try:
        uploaded = files.upload()
        if not uploaded:
            raise FileNotFoundError("No file was uploaded. Please try again.")
        print("Video uploaded successfully.")
        return list(uploaded.keys())[0]
    except Exception as e:
        print(f"An error occurred during upload: {e}")
        return None

# Step 2: Extract frames with video name and timestamp in filename
def extract_frames(video_path, output_folder, interval_sec):
    print("Extracting frames...")
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    if not fps or fps <= 0:
        raise ValueError("Could not determine video FPS.")

    # Extract base video name without extension for use in filenames
    video_base_name = os.path.splitext(os.path.basename(video_path))[0]

    frame_interval = int(fps * interval_sec)
    frame_count = 0
    saved_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            timestamp_sec = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
            timestamp_clean = f"{timestamp_sec:.2f}"
            frame_filename = f"{video_base_name}__frame_{saved_count:04d}__{timestamp_clean}.jpg"
            frame_path = os.path.join(output_folder, frame_filename)
            cv2.imwrite(frame_path, frame)
            saved_count += 1

        frame_count += 1

    cap.release()
    print(f"Saved {saved_count} frames to {output_folder}.")

# Step 3: Zip frames
def zip_frames(folder_path, zip_path):
    print(f"Zipping frames into {zip_path}...")
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files_in_dir in os.walk(folder_path):
            for file in files_in_dir:
                full_path = os.path.join(root, file)
                zipf.write(full_path, arcname=file)
    print("Zipping complete.")

# Step 4: Download zip
def download_zip(zip_path):
    print(f"Preparing {zip_path} for download...")
    files.download(zip_path)

# Full pipeline
def run_pipeline():
    uploaded_filename = upload_video()
    if uploaded_filename:
        try:
            extract_frames(uploaded_filename, output_dir, frame_interval_seconds)
            zip_frames(output_dir, zip_output)

            # Clean up
            os.remove(uploaded_filename)
            shutil.rmtree(output_dir)

            download_zip(zip_output)

        except Exception as e:
            print(f"An error occurred: {e}")
            # Clean up in case of an error to prevent partial files
            if os.path.exists(output_dir):
                shutil.rmtree(output_dir)
            if os.path.exists(zip_output):
                os.remove(zip_output)
            if os.path.exists(uploaded_filename):
                os.remove(uploaded_filename)

# Run it
run_pipeline()

Please choose your video file to upload.


Saving Billy-Hargrove-working-out-[Stranger-Things-Season 2].mp4 to Billy-Hargrove-working-out-[Stranger-Things-Season 2].mp4
Video uploaded successfully.
Extracting frames...
Saved 197 frames to temp_frames.
Zipping frames into extracted_frames.zip...
Zipping complete.
Preparing extracted_frames.zip for download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>